# Face recognition to automate AWS

In [ ]:
# import cv2
# import numpy as np

# # HERE THE HAAR FACE CLASSIFIER IS LOADING
# face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# # fUNCTION TO DETECT THE FACE AND CROP THAT
# def face_extractor(img):
    
#     img_in_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     faces = face_classifier.detectMultiScale(img_in_gray, 1.3, 5)
    
#     if faces is ():
#         return None
    
#     # CROPPING THE FACE PART
#     for (x,y,w,h) in faces:
#         cropped_face = img[y:y+h, x:x+w]

#     return cropped_face

# # WEBCAM INITIALIZATION
# cap = cv2.VideoCapture(0)
# count = 0

# # Collect 100 samples of your face from webcam input
# while True:

#     ret, frame = cap.read()
#     if face_extractor(frame) is not None:
#         count += 1
#         face = cv2.resize(face_extractor(frame), (200, 200))
#         face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

#         # SAVING EACH IMAGE IS A SPECIFIC FOLDER "M1"
#         file_name_path = './TASK6/AWS/' + str(count) + '.jpg'
#         cv2.imwrite(file_name_path, face)

#         # Put count on images and display live count
#         cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
#         cv2.imshow('FACES IS CROPPING', face)
        
#     else:
#         print("YOUR FACE IS NOT VISIBLE PROPERLY")
#         pass

#     if cv2.waitKey(1) == 13 or count == 200:
#         break
        
# cap.release()
# cv2.destroyAllWindows()      
# print("Collecting Samples Complete")

# Model Training using the captured image of a Person B

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# TAKING THE DATA FOR TRAINING PURPOSE FROM "M1" FOLDER
path = './TASK6/M1/'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

# LIST OF DATA AND LABELS
Training_Data = []
Labels = []

# OPEN TRAINING IMAGES IN OUR PATH AND CREATING A NUMPY ARRAY FOR TRAINING DATA
for i, files in enumerate(onlyfiles):
    image_path = path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# CREATE A NUMPY ARRAY FOR BOTH TRAINING DATA AND LABELS
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

aws_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
aws_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully!!!")


Model trained sucessefully!!!


# Face detection and launching an AWS instance, creating 5 GB EBS volume and attaching this volume with the instance


In [2]:
import cv2
import numpy as np
import os
import time

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#AWS TASK
def aws():
    #LAUNCHING AN INSTANCE
    os.popen("aws ec2 run-instances --image-id ami-011c99152163a87ae --count 1 --instance-type t2.micro --key-name sikey --subnet-id subnet-50a85c3b --security-group-ids sg-68e62114 --tag-specifications ResourceType=instance,Tags=[{Key=Name,Value=DM}]")
    print("AWS instance launched!!!\n")
    time.sleep(15)
    
    #CREATING A 5GB VEBS VOLUME
    os.popen("aws ec2 create-volume --availability-zone ap-south-1a --volume-type gp2 --size 5 --tag-specifications ResourceType=volume,Tags=[{Key=Name,Value=DM}]")
    print("AWS volume of 5GB created!!!\n")
    time.sleep(15)
    
    #QUERY FOR CREATED VOLUME_ID
    volume_id = os.popen('aws ec2 describe-volumes --filters Name=status,Values=available Name=availability-zone,Values=ap-south-1a --query Volumes[*].VolumeId --output text').read()
    volume_id = volume_id[0:len(volume_id)-1]
    print("The volume id of 5 GB EBS volume ",volume_id)
    time.sleep(15)
    
    #QUERY FOR RUNNING INSTANCE_ID
    instance_id = os.popen('aws ec2 describe-instances --filters Name=instance-state-name,Values=running --query "Reservations[*].Instances[*].InstanceId" --output text').read()
    instance_id = instance_id[0:len(instance_id)-1]
    print("The current running instance id ", instance_id)
    time.sleep(15)
    
    #ATTACHING THE 5 GB OF VOLUME TO THE INSTANCE
    os.popen('aws ec2 attach-volume --volume-id {0} --instance-id {1} --device /dev/sdf'.format(volume_id, instance_id))
    
    print("TASK 6 DONE!!!")


def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# WEBCAM INITIALIZATION
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # PASSING FACE FOR THE PREDICTION
        # "RESULTS" IS A TUPLE COMPRISES OF LABELS AND CONFIDENCE SCORE
        results = aws_model.predict(face)
        
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + 'FACE IS MATCHING!!!'
            cv2.putText(image, display_string, (50, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255,0), 2)
        
        if confidence > 90:
            cv2.putText(image, "FACE MATCHED!!!", (100, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('FACE RECOGNITION', image )
            time.sleep(5)
            aws()
            break
            
        else:
            cv2.putText(image, "FACE IS NOT MATCHING by 90%!!!", (100, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
            cv2.imshow('FACE RECOGNITION', image )

    except:
        cv2.putText(image, "FACE DOESN'T FOUND!!!", (50, 100) , cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
        cv2.putText(image, "PlEASE LOOK AT CAMERA!!!", (50, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
        cv2.imshow('FACE RECOGNITION', image )
        pass
        
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()     

<>:43: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:43: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-2-0c342226acaa>:43: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


AWS instance launched!!!

AWS volume of 5GB created!!!

The volume id of 5 GB EBS volume  vol-03dc73ac73ed3dae1
The current running instance id  i-0458b309701c95a15
TASK 6 DONE!!!


In [ ]:
cap.release()